# Azure AI Agents s podporou Model Context Protocol (MCP) - Python

Tento notebook ukazuje, jak používat Azure AI Agents s nástroji Model Context Protocol (MCP) v Pythonu. Demonstruje, jak vytvořit inteligentního agenta, který může využívat externí MCP servery (například Microsoft Learn) pro rozšířené schopnosti pomocí autentizace bez klíče.


## Instalace potřebných balíčků Pythonu

Nejprve je nutné nainstalovat potřebné balíčky Pythonu:
- **azure-ai-projects**: Základní SDK pro Azure AI Projects
- **azure-ai-agents**: SDK pro Azure AI Agents pro vytváření a správu agentů
- **azure-identity**: Poskytuje bezklíčovou autentizaci pomocí DefaultAzureCredential
- **mcp**: Implementace Model Context Protocol pro Python


## Výhody autentizace bez klíčů

Tento notebook demonstruje **autentizaci bez klíčů**, která přináší několik výhod:
- ✅ **Žádné API klíče k spravování** - Využívá autentizaci založenou na identitě Azure
- ✅ **Zvýšená bezpečnost** - Žádná tajemství uložená v kódu nebo konfiguračních souborech
- ✅ **Automatická rotace přihlašovacích údajů** - Azure se stará o správu životního cyklu přihlašovacích údajů
- ✅ **Řízení přístupu na základě rolí** - Používá Azure RBAC pro detailní nastavení oprávnění
- ✅ **Podpora více prostředí** - Funguje bez problémů jak v prostředí vývoje, tak v produkci

`DefaultAzureCredential` automaticky vybírá nejlepší dostupný zdroj přihlašovacích údajů:
1. **Spravovaná identita** (při běhu v Azure)
2. Přihlašovací údaje z **Azure CLI** (během lokálního vývoje)
3. Přihlašovací údaje z **Visual Studio**
4. **Proměnné prostředí** (pokud jsou nastaveny)
5. Autentizace přes **interaktivní prohlížeč** (jako záložní možnost)


## Nastavení autentizace bez klíčů

**Předpoklady pro autentizaci bez klíčů:**

### Pro lokální vývoj:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Pro prostředí Azure:
- Aktivujte **System-assigned Managed Identity** na vašem Azure zdroji
- Přiřaďte odpovídající **RBAC role** k této identitě:
  - `Cognitive Services OpenAI User` pro přístup k Azure OpenAI
  - `AI Developer` pro přístup k projektům Azure AI

### Proměnné prostředí (volitelné):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Žádné API klíče ani připojovací řetězce nejsou potřeba!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Importujte potřebné knihovny

Importujte nezbytné Python moduly:
- **os, time**: Standardní Python knihovny pro proměnné prostředí a zpoždění
- **AIProjectClient**: Hlavní klient pro Azure AI Projects
- **DefaultAzureCredential**: Bezklíčová autentizace pro Azure služby
- **Třídy související s MCP**: Pro vytváření a správu MCP nástrojů a zpracování schválení


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Nastavení konfigurace MCP serveru

Nastavte konfiguraci MCP serveru pomocí proměnných prostředí s výchozími hodnotami:
- **MCP_SERVER_URL**: URL adresa MCP serveru (výchozí je Microsoft Learn API)
- **MCP_SERVER_LABEL**: Označení pro identifikaci MCP serveru (výchozí je "mslearn")

Tento přístup umožňuje flexibilní konfiguraci napříč různými prostředími.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Vytvoření klienta Azure AI Project (Autentizace bez klíče)

Inicializujte klienta Azure AI Project pomocí **autentizace bez klíče**:
- **endpoint**: URL koncového bodu projektu Azure AI Foundry
- **credential**: Používá `DefaultAzureCredential()` pro bezpečnou autentizaci bez klíče
- **Žádné API klíče nejsou potřeba**: Automaticky vyhledá a použije nejlepší dostupné přihlašovací údaje

**Proces autentizace:**
1. Kontroluje Managed Identity (v prostředí Azure)
2. Přechází na přihlašovací údaje Azure CLI (pro lokální vývoj)
3. Používá další dostupné zdroje přihlašovacích údajů podle potřeby

Tento přístup eliminuje potřebu spravovat API klíče nebo připojovací řetězce ve vašem kódu.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Vytvoření definice nástroje MCP

Vytvořte nástroj MCP, který se připojuje k serveru Microsoft Learn MCP:
- **server_label**: Identifikátor pro server MCP
- **server_url**: URL endpoint serveru MCP
- **allowed_tools**: Volitelný seznam pro omezení, které nástroje mohou být použity (prázdný seznam umožňuje všechny nástroje)

Tento nástroj umožní agentovi přístup k dokumentaci a zdrojům Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Vytvoření agenta a vedení konverzace (bez klíčů)

Tato komplexní sekce ukazuje kompletní **workflow agenta bez klíčů**:

1. **Vytvoření AI agenta**: Nastavte agenta s modelem GPT-4.1 nano a nástroji MCP
2. **Vytvoření vlákna**: Založte konverzační vlákno pro komunikaci
3. **Odeslání zprávy**: Zeptejte se agenta na rozdíly mezi Azure OpenAI a OpenAI
4. **Schvalování nástrojů**: Automaticky schvalujte volání nástrojů MCP, pokud je to potřeba
5. **Sledování průběhu**: Sledujte pokrok agenta a řešte případné požadované akce
6. **Zobrazení výsledků**: Zobrazte detaily konverzace a použití nástrojů

**Funkce bez klíčů:**
- ✅ **Žádná pevně zakódovaná tajemství** - Veškeré ověřování je zajištěno identitou Azure
- ✅ **Bezpečné jako standard** - Používá řízení přístupu na základě rolí
- ✅ **Zjednodušené nasazení** - Není nutná správa přihlašovacích údajů
- ✅ **Přátelské k auditu** - Veškerý přístup je sledován prostřednictvím identity Azure

Agent bude používat nástroje MCP k přístupu k Microsoft Learn zdrojům s plnou bezpečností a bez správy API klíčů.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). Ačkoli se snažíme o přesnost, mějte prosím na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
